# TUTORIAL:
# Julia for technical computing

# Chapter 1: Language syntax

In order to gain some understanding of the notions of types and multiple dispatch in Julia, we are going to have to cover at least some basics. If you actually want to learn Julia and write Julia code, it would be a good idea to read the [manual](http://docs.julialang.org/en/release-0.4/manual/) (from start to finish).

## 1. First things first: variables

Use variables like you would in Matlab or Python. You don't declare any types, but Julia will always deduce them (and if it can't, the type of your variable will be `Any`). Read all about variables [here](http://docs.julialang.org/en/release-0.4/manual/variables/).

In [ ]:
x = 33

In [ ]:
y = 2.0

In [ ]:
z = "Hello, I am a string."

In [ ]:
a = 4 // 2    # this is a Julia-style rational number

In [ ]:
[1 2 3 4]   # a row vector

In [ ]:
[1, 2, 3, 4]   # and a column vector

In [ ]:
[1 2 3 4; 5 6 7 8]    # and a Matlab-style matrix

In [ ]:
[1; 2; 3; 4]

Julia has a familiar REPL interface (Read-Eval-Print-Loop) like Matlab and Python. Write code and Julia answers, for example:

In [ ]:
x+y

In [ ]:
x*y/2

In [ ]:
typeof(x)

### Variables are always references!

This is important to note from the start: variables in Julia are just names associated with a value. This means we always have **reference semantics**! This is (very) different from Matlab.

For example:

In [ ]:
a = [1, 2, 3, 4]

We introduce a variable `b` and assign `a` to it: this just means that `a` and `b` are references to the same data. Modifying the data via `b` will modify the data via `a` too:

In [ ]:
b = a
b[2] = 5
b

In [ ]:
a

You can avoid reference semantics with an explicit copy:

In [ ]:
c = copy(b)
c[2] = 7
c

In [ ]:
b

Since arrays are always passed by references, it becomes easy to make functions that modify their arguments - you can make **in-place algorithms**. Note for future reference the Julia convention of naming functions that modify one of their arguments in a particular way, namely ending in an exclamation point. For example, [see](http://docs.julialang.org/en/release-0.4/stdlib/collections/?highlight=push!#Base.push!) the `push!` function that changes a vector by adding an element to it.

More importantly, reference semantics lets you avoid making unnecessary copies of data all over the place. Matlab avoids the copying using a copy-on-write mechanism. This is a valid choice to make, but it is still less efficient than just treating everything as a reference.

"Simple" values like integers do not have reference semantics. Consider:

In [ ]:
a = 4
b = a
b = 5

In [ ]:
a

In [ ]:
b

Modifying `b` did not modify `a`. Why? Because we did not actually modify `b` at all! The line `b = 5` simply made `b` refer to a different integer, unrelated to what it was pointing at before. It is a new assignment, no different from the line `a = 4` above.

In any case, you can not change the value of an integer, because it is **immutable**. Arrays are mutable, in the sense that you can alter the third entry of an array. This does not make integers special in any way. Julia has mutable and immutable types, we will see that later.

## 2. More on vectors and matrices

Read more about [arrays](http://docs.julialang.org/en/release-0.4/manual/arrays/) and [linear algebra](http://docs.julialang.org/en/release-0.4/stdlib/linalg/) in the manual. Beware that some aspects of arrays are going to change, rather significantly, in Julia version 0.5. The current goals (at the time of writing) are described on [GitHub](https://github.com/JuliaLang/julia/issues/13157). In particular, in Julia up to version 0.4, indexing into an array creates a copy of data. This is a notorious reason for unexpected slowness. Starting with version 0.5, the default will be to create a view, like it is the default in Numpy. This changes the semantics: modifying data through a view will modify the data of the underlying array. That is why the upcoming release of 0.5 is already referred to as the *Arraypocalypse*. 

There has also been a *tuplocalypse* before when [redesigning tuples](https://github.com/JuliaLang/julia/pull/10380). It is expected that in the future there will be fewer disruptive and backwards incompatible changes. For the time being, **design changes that ultimately result in better performance are considered acceptable** in the Julia community, at least until version 1.0 is released. Whenever that may be, don't ask around for a timeframe. It will be there when it's done. (And if you have a vested interest, there are many ways to help make it happen quicker.)

### This is very convenient: list comprehension

In [ ]:
a = [factorial(i) for i=1:10]

In [ ]:
A = [1/(i+j) for i=1:4,j=1:4]

In the previous two examples I have used [*list comprehension*](http://docs.julialang.org/en/release-0.4/manual/arrays/?highlight=comprehension#comprehensions), a convenient way for creating lists or vectors. Note that, unlike in NumPy, **lists and arrays are the same thing in Julia**: there is a single `Array` type.

### Linear algebra

Julia links to several well known libraries for numerical computations, including BLAS and LAPACK. All the usual suspects are present. See the manual on [Linear Algebra](http://docs.julialang.org/en/release-0.4/stdlib/linalg/#stdlib-linalg) methods in the standard library and the available [Matrix Factorizations](http://docs.julialang.org/en/release-0.4/manual/linear-algebra/) for a more complete picture. Here are some examples.

In [ ]:
det(A)

In [ ]:
qr(A)

In [ ]:
svd(A)

In [ ]:
@which svd(A)

Looking at the code in linalg/svd.jl, Julia's `svd` seems to eventually call LAPACK's [gesdd](https://github.com/JuliaLang/julia/blob/e5c6964a497a71fb940117530c1867ddd71f4c67/base/linalg/svd.jl#L17) routine in this case. You can call LAPACK routines directly yourself, if you are so inclined. In general, there is very little overhead in [calling external C or Fortran functions](http://docs.julialang.org/en/release-0.4/manual/calling-c-and-fortran-code/).

### Elementwise operations

There are `.*` and `.^` operators in Julia, like in Matlab, to perform element-wise operations.

In [ ]:
B = [1 2; 3 4]

In [ ]:
B.*B

In [ ]:
B.^2

As in Matlab, these are different from the standard Matrix multiplication and exponentiation.

In [ ]:
B*B

In [ ]:
B^2

In fact, Julia supports many more operators, including many unicode symbols. See the list of exported symbols in the souce code [base/operators.jl](https://github.com/JuliaLang/julia/blob/master/base/operators.jl#L458).

For writing fancy mathematics, you may want to consider using an editor that has Unicode support. In IPython, you can type \in followed by a TAB and you will get:

In [ ]:
∈

This is just a function, called `in`. You can redefine all operators to do whatever you want with your own types - read on for the good stuff later.

In [ ]:
in(5, [3 5])

In [ ]:
5 ∈ [3 5]

## 3. Functions

### Syntax of functions

Functions are a bit like in Matlab, except that you don't specify output variables. The last expression that is evaluated yields the return value (like it is in, say, Maple).

In [ ]:
function fibonacci(n)
    if (n == 1) || (n==0)
        1
    else
        fibonacci(n-1) + fibonacci(n-2)
    end
end

In [ ]:
fibonacci(5)

In [ ]:
[fibonacci(i) for i = 0:10]

There is a shorthand notation for short functions, of the form f(x) = do_something_with_x. This is closer to mathematics:

In [ ]:
sqr(x) = x*x

In [ ]:
sqr(5)

In [ ]:
sqr(5.0)

You can also explicitly write `return` if you like

In [ ]:
function my_maximum(x,y)
    if x > y
        return x
    else
        return y
    end
end

In [ ]:
my_maximum(2,3)

I can't resist adding that the following definition is shorter, using a ? b : c shorthand notation for if a then b else c.

In [ ]:
my_maximum(x,y) = (x>y) ? x : y

Finally, you can create anonymous functions.

In [ ]:
x -> cos(x)

You can pass around functions as arguments, including operators (which are really just functions).

In [ ]:
composite(f, g, x) = f(g(x))

In [ ]:
composite(cos, sin, 0.2)

Think about this one :-)

In [ ]:
composite(-, -, 1)

### Each function call leads to a specialized compiled version of the function

Let's make another more insightful remark. We've seen that Julia keeps track of types: 5 is an Int64, 5.0 is a Float64. When you invoke a function with a number of arguments, **Julia compiles a version of the function that is specific to the types of the arguments.** This results in maximum performance. I have given only one definition of the `sqr` function, yet by now the compiler has made two separate versions: one for Int64, one for Float64.

If you are curious, you can inspect the resulting machine instructions on your machine. You don't have to. Nobody expects you to. But you *can*:

In [ ]:
@code_native sqr(5)

In [ ]:
@code_native sqr(5.0)

These instructions are quite short. The machine instructions that are used to execute your command are specific to the arguments that you have given. They are different for Int64 and for Float64.

You don't usually specify types of variables, but Julia tries to deduce them (through *type inference*). If Julia succeeds in doing that at compile-time, then your code will be pretty much as fast as it would be in C. If a type can not be inferred without ambiguity at compile-time, a runtime check happens and this slows things down a bit. But regardless of when Julia learns about the type of your variable, at compile-time or at runtime, the *same function* ends up getting called.

### More introspective features

Julia offers more [introspective features](http://blog.leahhanson.us/julia-introspects.html). The compilation process consists of the following steps:
* lowering: your code is first transformed into a so-called Abstract Syntax Tree (AST), a hierarchical structure that encodes your function body. This tree is immediately *lowered* to a lower level of instructions and represented in something that looks like a LISP structure (the functional language). The AST and lowered AST step can be relevant if you are writing [macros](http://docs.julialang.org/en/release-0.4/manual/metaprogramming/).
* type inference: the compiler tries to infer the types of all variables by analyzing your code
* llvm: the typed, lowered and optimized AST tree is transformed into a set of LLVM instructions. [LLVM](http://llvm.org/) is an open-source just-in-time compiler.
* native instructions: the LLVM instructions are converted into machine instructions for your specific machine.

You can ask for the result in each of these steps. Again, you don't *have to*, but you *can*.

In [ ]:
@code_lowered sqr(5)   # show the lowered AST tree

In [ ]:
@code_typed sqr(5)     # the AST tree, optimized and augmented with type information
# note the return type on the last line, following end

In [ ]:
@code_llvm sqr(5)      # LLVM instructions

In [ ]:
@code_native sqr(5)     # native assembler code

## 4. Control flow

All the usual suspects are [available](http://docs.julialang.org/en/release-0.4/manual/control-flow/).

In [ ]:
for i = 1:10
    println(i)
end

In [ ]:
i = 0
while (i < 10)
    i += 1
    println(i)
end

Note that `1:10` creates a Range object, which is iterable. Using "`for in`" you can iterate over any iterable object. See the [manual](http://docs.julialang.org/en/release-0.4/manual/interfaces/#iteration) for details.

In [ ]:
1:10

In [ ]:
typeof(1:10)

In [ ]:
for i in 1:10
    println(i)
end

In [ ]:
for i in [1,4,0]
    println(i)
end

In [ ]:
if 2 < 3
    println("Hey, I was expecting that.")
else
    println("This is just to illustrate the syntax.")
end

## 5. Exercises

I said you wouldn't actually learn Julia, and yet here are some exercises... sorry about that. Feel free to skip, come back later, try your own exercises or go outside for a run. It is always nice to be in good shape.

1) Write a function called `twosum`, that returns two times the sum of the elements of a vector.

In [ ]:
function twosum(x)
    # code goes here
end

In [ ]:
twosum([1 2 3])

2) Consider the following matrix:

In [ ]:
A = cos([1; 2; 3]) * sin([4 5 6])

Write a function called `largenumbers` that loops over all elements of a given matrix and prints each element that is greater than `0`, or greater than a given threshold t. Test it on the matrix above.

In [ ]:
function largenumbers(A)
    # code goes here. Test whether elements are great than zero, and print them ('println(a)')
end

In [ ]:
function largenumbers(A, threshold)
    # code goes here
end